In [343]:
from gurobipy import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from datetime import timedelta
from operator import itemgetter 
import time

In [678]:
data = pd.read_csv('Dataset_goal_3_time_csv.csv',sep=';')

selectedDay = 'Sunday'

### Calcuate average travel time from max and min travel time
city = list()
highway = list()
for row in np.arange(data.shape[0]):
    #print('Scenario I average: ',sum(list(data.iloc[row,3:5].values))/2)
    city.append(sum(list(data.iloc[row,3:5].values))/2)
    
    #print('Scenario II average: ',sum(list(data.iloc[row,6:].values))/2)
    highway.append(sum(list(data.iloc[row,6:].values))/2)
    
data['averageTimeCity'] = city
data['averageTimeHighway'] = highway

### Split Columns

daySplitted = data.Day.str.split(' - ', n = 1, expand=True)

data['Weekday'] = daySplitted[0]
data['Date'] = daySplitted[1]

dateSplitted = data.Date.str.split('/',n=2, expand = True)

data['Month'] = dateSplitted[0]
data['Day'] = dateSplitted[1]
data['Year'] = dateSplitted[2]

timeSplitted = data['Departure time'].str.split(':',n=1, expand=True)

data['Hour'] = timeSplitted[0]
data['Minute'] = timeSplitted[1]


days = list(data.Weekday.drop_duplicates())
    
### tuples(scenario, weekday, year, month, day, hour, minute | trip time)

tupledict_input = list()
N=10
for d in days:
    tempData = data[data.Weekday == d]
    for st in range(len(tempData)):
        startingTime = tempData['Departure time'].values[st]
        averageTravelTimeCity = tempData['averageTimeCity'].values[st]
        averageTravelTimeHighway = tempData['averageTimeHighway'].values[st]
        tupledict_input.append([('city', d, tempData.iloc[st,:].Year,tempData.iloc[st,:].Month,
                                tempData.iloc[st,:].Day,tempData.iloc[st,:].Hour,tempData.iloc[st,:].Minute), 
                                averageTravelTimeCity])
        tupledict_input.append([('highway', d, tempData.iloc[st,:].Year,tempData.iloc[st,:].Month,
                                tempData.iloc[st,:].Day,tempData.iloc[st,:].Hour,tempData.iloc[st,:].Minute), 
                                averageTravelTimeHighway])

times = tupledict(tupledict_input) 
relevantTimes = times.subset('*', selectedDay)
min_value = min(relevantTimes.select())
min_day = [i for i, x in enumerate(relevantTimes) if x == min_value]

### calculate arrival time for each trip with min time
arrivalTime = list()

for t in relevantTimes.iteritems():
    if t[1] == min_value:
        arrivalTime.append(datetime(int(t[0][2]),int(t[0][3]),int(t[0][4]),
                             int(t[0][5]),int(t[0][6])) + timedelta(minutes = min_value))

goalTime = 10 * 60
arrivalTimeMinutes = [arrivalTime[x].hour * 60 + arrivalTime[x].minute for x in range(len(arrivalTime))]

### variable to choose only the minimum driving time

choice = list()
for x in range(len(relevantTimes)):
    choice.append(m.addVar(vtype=GRB.BINARY))
    
temp = relevantTimes.select()
    
m.update()

m.addConstr(quicksum(choice) == 1)

m.setObjective(quicksum(temp[x] * choice[x] for x in range(len(temp))), GRB.MINIMIZE)

m.optimize()

Optimize a model with 74 rows, 28 columns and 28 nonzeros
Variable types: 0 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives (1 combined) ...
---------------------------------------------------------------------------
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (weighted) ...
---------------------------------------------------------------------------

Optimize a model with 74 rows, 28 columns and 28 nonzeros
Variable types: 0 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 56 ro

C:\Users\yubim\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
